In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark')

import math
import openmc
import openmc.mgxs as mgxs
import openmc.data
import random
import h5py

In [2]:
class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.get_virtual_cs()
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs for cross_section in self.sig_t]
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
        
        
    
        

In [3]:
cs_fission_pu_239 = [0.081600]

cs_fission_h2o = [0.0]

cs_capture_pu_239 = [0.019584]

cs_capture_h2o = [0.032640]

cs_scattering_pu_239 = [0.225216]

cs_scattering_h2o = [0.293760]

cs_total_pu_23 = [0.32640]

cs_total_h2o = [0.32640]

cs_production_neutrons_pu_239 = [3.24]
cs_production_neutrons_h2o = [0.0]

cs_c_value_pu_239 = [1.50]
cs_c_value_h2o = [0.90]

pu_23_cs = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)


h2o_cs = Sig(1, cs_fission_h2o, cs_capture_h2o, cs_scattering_h2o, 
                     cs_c_value_pu_239,  cs_total_h2o)

pu_h2o_CS_table_2 = {  
    "pu_23" : pu_23_cs,
    "h2o" : h2o_cs    
}

energy_groups_2 = [0, 2 * 1E10]

In [4]:
class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z


In [5]:
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)
        

In [6]:
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0. 
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        
        
        

In [7]:
## Infinite cylinder parallel to z-axis

class ZCylinder:
    def __init__(self, x_0, y_0, r):
        self.x_0 = x_0
        self.y_0 = y_0
        self.r = r
        
    def distance(self, particle):
    
        a_ = particle.direction.tetta_x * particle.direction.tetta_x + (particle.direction.tetta_y * particle.direction.tetta_y )
        
        ## If a=0a 0a=0, this means the particle is parallel to the cylinder and will thus never intersect it
        if (abs(a_) < 1e-9):
            return -1
        
        c_ = ((particle.coordinates.x - self.x_0) * (particle.coordinates.x - self.x_0) + 
              (particle.coordinates.y - self.y_0) * (particle.coordinates.y - self.y_0) - self.r * self.r)

        k_ = (particle.coordinates.x -self.x_0) * particle.direction.tetta_x + (particle.coordinates.y -self.y_0) * particle.direction.tetta_y
        
        dec = k_ * k_ - a_ * c_
        
        if dec < 0:
            return -1
        
        dist_1 = (-k_ + math.sqrt(k_ * k_ - a_ * c_))/a_
        dist_2 = (-k_ - math.sqrt(k_ * k_ - a_ * c_))/a_
         
        distance = min(dist_1, dist_2)
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    
    def get_reflected_direction(self, angle):
        
        eps = random.uniform(0, math.pi * 2)
        tetta = random.uniform(0, math.pi)
        r_reflected = Direction(eps, tetta)
        
        return r_reflected  
    
    def get_sign(self, particle):
            
        sign = ((self.x_0 - particle.coordinates.x) * (self.x_0 - particle.coordinates.x) + 
        (self.y_0 - particle.coordinates.y) * (self.y_0 - particle.coordinates.y) - self.r * self.r)
        
        if sign == 0:
            return 0
            
        if sign < 0:
            return -1
        else:
            return 1  
    

In [8]:
class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    
    def get_reflected_direction(self, angle):
        normal_sq = self.A * self.A + self.B * self.B + self.C * self.C
        
        x_numerator = 2 * self.A * (self.A * angle.tetta_x + self.B * angle.tetta_y +
                                 self.C * angle.tetta_z)
        
        y_numerator = 2 * self.B * (self.A * angle.tetta_x + self.B * angle.tetta_y +
                                 self.C * angle.tetta_z)
        
        z_numerator = 2 * self.C * (self.C * angle.tetta_x + self.B * angle.tetta_y +
                                 self.C * angle.tetta_z)
        
        
        r_reflected = Direction(0, 0)      
      #  print(str(angle.tetta_x) + " " + str(angle.tetta_y) + " " + str(angle.tetta_z))
       # print(str(x_numerator / normal_sq) + " " + str(y_numerator / normal_sq) + " " + str(z_numerator / normal_sq))
        r_reflected.tetta_x = angle.tetta_x - x_numerator / normal_sq
        r_reflected.tetta_y = angle.tetta_y - y_numerator / normal_sq
        r_reflected.tetta_z = angle.tetta_z - z_numerator / normal_sq
        
       # print(str(r_reflected.tetta_x) + " " + str(r_reflected.tetta_y) + " " + str(r_reflected.tetta_z))
        
        return r_reflected
    
    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1    
        

In [9]:
class Cell:
    def __init__(self):
        self.surfaces = []
        self.signs = []
        self.size = 
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
        
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside  
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = self.surfaces[i].distance(particle)
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
            
            

In [43]:
class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if not cell.is_inside(particle):
                return xs[i]
            
    def 
            

In [10]:
x_5 = Plane(0, 1, 0, 30)
x_m5 = Plane(0, 1, 0, -30)

y_5 = Plane(1, 0, 0, 30)
y_m5 = Plane(1, 0, 0, -30)



z_5 = Plane(0, 0, 1, 30)
z_m5 = Plane(0, 0, 1, -30)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]
signs = [-1, +1, -1, +1, -1, +1]
box = Cell(surfaces, signs)


particle_inside = Particle()

particle_inside.set_coordinates(1, 1, 1)
particle_inside.set_direction(0.5, 0.6)
box.is_inside(particle_inside)

True

In [11]:
import random

def make_initial_sources(number_of_paricles, box_size, energy=10.0e6):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                eps = random.uniform(0, math.pi * 2.)
                tetta = random.uniform(0, math.pi * 2.)
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
             #   current_particle.print_coordinates()
               # print(" eps  "+ str(eps))
                #print(" tetta  "+ str(tetta))
                current_particle.set_direction(eps, tetta)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources


In [12]:
plane_1 = Plane(1, 0, 0, 5)
plane_2 = Plane(1, 0, 0, -5)

plane_3 = Plane(0, 1, 0, 5)
plane_4 = Plane(0, 1, 0, -5)

plane_5 = Plane(0, 0, 1, 5)
plane_6 = Plane(0, 0, 1, -5)

surfaces = [plane_1, plane_2, plane_3, plane_4, plane_5, plane_6]
signs = [-1, +1, -1, +1, -1, +1]
box = Cell(surfaces, signs)

In [13]:
particle_inside = Particle()

particle_inside.set_coordinates(1, 1, 1)
particle_inside.set_direction(0.5, 0.6)
box.is_inside(particle_inside)

True

In [14]:
test_cylinder = ZCylinder(1, 5, 5)

plane_5 = Plane(0, 0, 1, 5)
plane_6 = Plane(0, 0, 1, -5)

surfaces = [test_cylinder, plane_5, plane_6]
signs = [-1, -1, +1]

cylinder = Cell(surfaces, signs)

In [15]:
particle_inside = Particle()

particle_inside.set_coordinates(1, 1, 1)
particle_inside.set_direction(0.5, 0.6)
cylinder.is_inside(particle_inside)

True

In [16]:
particle_outside = Particle()

particle_outside.set_coordinates(9, 9, 9)
particle_outside.set_direction(0.5, 0.6)
box.is_inside(particle_outside)

False

In [17]:
def get_free_path(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = c_s.sig_t[energy_group_idx]#
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  

In [18]:
def move_particle(particle, t):
    new_x = particle.direction.tetta_x * (t - 0.0001) + particle.coordinates.x
    new_y = particle.direction.tetta_y * (t - 0.0001) + particle.coordinates.y
    new_z = particle.direction.tetta_z * (t - 0.0001) + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)

In [19]:
def get_temp_coordinates(particle, t):
    temp_particle = Particle()
    new_x = particle.direction.tetta_x * (t) + particle.coordinates.x
    new_y = particle.direction.tetta_y * (t) + particle.coordinates.y
    new_z = particle.direction.tetta_z * (t) + particle.coordinates.z
    temp_particle.set_coordinates(new_x, new_y, new_z)
    return temp_particle
    

In [20]:
def is_collision_virual(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    virtual_cs = c_s.virtual[energy_group_idx]
    total_cs = c_s.sig_t[energy_group_idx]
    if virtual_cs / total_cs >= random_number:
        return True
    else:
        return False

In [21]:
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)
    

In [22]:
def set_random_direction_vector(particle):
    eps = random.uniform(0, math.pi * 2)
    tetta = random.uniform(0, math.pi)
    particle.set_direction(eps, tetta)

In [23]:
def process_real_collision(particle, free_path, universe, c_s):
    
    energy_group_idx = particle.get_energy_group()
  
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    particle.add_path(free_path)
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction_vector(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()

In [24]:
def reflect_particle(particle, universe):
    distance, surface_idx = universe.get_minimum_distance(particle)
   # print(" distance " + str(distance))
    move_particle(particle, distance)
    particle.direction = universe.surfaces[surface_idx].get_reflected_direction(particle.direction)
    
    
    

In [25]:
def process_one_particle_history(particle, universe, c_s):
    
    while not particle.is_terminated():
        
        free_path = get_free_path(particle, c_s)
        particle_temp = get_temp_coordinates(particle, free_path)
        if not universe.is_inside(particle_temp):
            reflect_particle(particle, universe)
        else:
            move_particle(particle, free_path)            
            
        if is_collision_virual(particle, c_s):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, universe, c_s)
            
    return particle

In [26]:
def process_one_particle_history_black_boundary(particle, universe, c_s):
    
    while not particle.is_terminated():
        
        free_path = get_free_path(particle, c_s)

        move_particle(particle, free_path)            
            
        if is_collision_virual(particle, c_s):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, universe, c_s)
            
    return particle

In [27]:
def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.0001:
            existing_particles.append(particle)
                     
    return existing_particles

In [28]:
def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

In [29]:
def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        particles[i].set_path(0.)
    

In [30]:
def calculate_k_effective(weights, batch_size):
    
    k_effective = sum(weights) / batch_size
    
    return k_effective

In [31]:
import copy

def splitting(particles):
    
    splitting_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        n_value = math.floor(particle.get_weight())
        if particle.get_weight() > 1.0 and n_value > 1:       
            random_value = random.uniform(0, 1)
            splitted_weight = particle.get_weight() / n_value
            if particle.get_weight() - n_value >= random_value:
                n_value +=1
            particle.set_weight(splitted_weight)
            for i in range(0, n_value - 1):
                splitting_particles.append(copy.deepcopy(particle))
        else:
            splitting_particles.append(particle)
    return splitting_particles

In [32]:
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights
                
    return particles

In [33]:
def make_particles_equal_weights(number_of_paricles, energy=10.0e6, box_size=[3., 3., 3.]):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
   
        eps = random.uniform(0, math.pi * 2.)
        tetta = random.uniform(0, math.pi * 2.)
        current_particle = Particle()
        current_particle.set_coordinates(x_coord + i * step_x, y_coord + i* step_y, z_coord + i * step_z)
      #  current_particle.print_coordinates()
       # print("eps  " + str(eps))
        #print("tetta  " + str(tetta))
        current_particle.set_direction(eps, tetta)
        current_particle.energy = energy
        current_particle.set_energy_groups(energy_groups_2)
        sources.append(current_particle)

    return sources

In [34]:
particles_equal_weights = make_particles_equal_weights(10, energy=10.0e6, box_size=[3., 3., 3.])

In [35]:
weights = get_weights(particles_equal_weights)
print(weights)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [36]:
def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate > random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])
    return particles_current

In [37]:
def flux_estimator(cell_volume, start_weight, particles_current_iteration, cross_section):

    
    sum_weighted_paths = 0.
    for i in range(0, len(particles_current_iteration)):
        particle = particles_current_iteration[i]
        energy_group_idx = particle.get_energy_group() 
  
        number_of_production_neutrons = cross_section.number_of_production_neutrons[energy_group_idx]   
        sum_weighted_paths += particles_current_iteration[i].get_weight()* (
            particles_current_iteration[i].get_path())
        
    total_flux = sum_weighted_paths/(cell_volume * start_weight)
    
    return total_flux
    

In [38]:
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            particle.set_weight(0.)


In [39]:
import statistics

def simulation_reflective_boundaries(universe, number_interations,  number_inactive, number_of_particles, c_s):
    
    k_effective_sample = []
    k_effective_std_sample = []
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.1, 0.1, 0.1])
    initial_size = len(initial_sources)
    current_k_effective = 0.
    weights_previous = [1.] * len(initial_sources)
    for i in range(0, number_interations):
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        

        batch_particles = []
        for j in range(0, batch_size):
            
            particle = initial_sources[j]
            terminate_particle = process_one_particle_history(particle, universe, c_s)
            batch_particles.append(terminate_particle)
   
        terminate_outside_particles(batch_particles, universe)
        batch_particles = russian_roulette(weights_previous, batch_particles)

        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        batch_particles = splitting(batch_particles)
        weights_cycle = get_weights(batch_particles)
        current_k_effective = calculate_k_effective(weights_cycle, initial_size)
        
        batch_particles = normalise_weights(batch_particles, initial_size)

        initial_sources = batch_particles
        current_weights = get_weights(batch_particles)
        weights_previous = current_weights

        if i > number_inactive:
            
            k_effective_sample.append(current_k_effective)
            
        if i > number_inactive + 1:
            k_effective_std = statistics.stdev(k_effective_sample)
            k_effective_std_sample.append(k_effective_std)
            print(" k effective  " + str(current_k_effective) + "+- " + str(k_effective_std))
        
            
    return k_effective_sample, k_effective_std_sample
    

In [40]:
x_5 = Plane(0, 1, 0, 5)
x_m5 = Plane(0, 1, 0, -5)

y_5 = Plane(1, 0, 0, 5)
y_m5 = Plane(1, 0, 0, -5)



z_5 = Plane(0, 0, 1, 5)
z_m5 = Plane(0, 0, 1, -5)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]
signs = [-1, +1, -1, +1, -1, +1]
box = Cell(surfaces, signs)

In [41]:
import statistics

def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s):
    
    k_effective_sample = []
    k_effective_std_sample = []
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.1, 0.1, 0.1])
    initial_size = len(initial_sources)
    current_k_effective = 0.
    weights_previous = [1.] * len(initial_sources)
    for i in range(0, number_interations):
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        

        batch_particles = []
        for j in range(0, batch_size):
            
            particle = initial_sources[j]
            terminate_particle = process_one_particle_history_black_boundary(particle, universe, c_s)
            batch_particles.append(terminate_particle)
   
     #   terminate_outside_particles(batch_particles, universe)
        batch_particles = russian_roulette(weights_previous, batch_particles)

        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        batch_particles = splitting(batch_particles)
        weights_cycle = get_weights(batch_particles)
        current_k_effective = calculate_k_effective(weights_cycle, initial_size)
        
        batch_particles = normalise_weights(batch_particles, initial_size)

        initial_sources = batch_particles
        current_weights = get_weights(batch_particles)
        weights_previous = current_weights

        if i > number_inactive:
            
            k_effective_sample.append(current_k_effective)
            
        if i > number_inactive + 1:
            k_effective_std = statistics.stdev(k_effective_sample)
            k_effective_std_sample.append(k_effective_std)
            print(" k effective  " + str(current_k_effective) + "+- " + str(k_effective_std))
        
            
    return k_effective_sample, k_effective_std_sample

In [149]:
particle_inside = Particle()

particle_inside.set_coordinates(-0.5, -0.5, -0.5)
particle_inside.set_direction(0., 3.14/2.)
box.is_inside(particle_inside)

True

In [125]:
box.get_minimum_distance(particle_inside)

(5.5, 4)

In [126]:
particle_inside.print_direction()

0.0 0.0 1.0


In [127]:
particle_inside.print_coordinates()

-0.5  -0.5 -0.5


In [128]:
reflect_particle(particle_inside, box)

In [129]:
particle_inside = Particle()

particle_inside.set_coordinates(-0.5, -0.5, -0.5)
particle_inside.set_direction(4.18524, 1.036991231)
slab_2.is_inside(particle_inside)

True

In [130]:
slab_2.get_minimum_distance(particle_inside)

(2.956300513716578, 5)

In [131]:
reflect_particle(particle_inside, slab_2)

In [132]:
particle_inside.print_direction()

-0.4397387863541912 -0.7440087070229503 1.3825489827982234


In [133]:
particle_inside.print_coordinates()

-1.7999560261213645  -2.6994389219108523 -1.9871799609437104


In [134]:
slab_2.get_minimum_distance(particle_inside)

(0.00010000000000024793, 4)

In [135]:
reflect_particle(particle_inside, slab_2)

In [136]:
particle_inside.print_coordinates()

-1.7999560261213645  -2.6994389219108523 -1.98717996094371


In [137]:
slab_2.get_minimum_distance(particle_inside)

(0.00010000000000024793, 5)

In [117]:
test_number_of_particles = 10
test_number_interations = 10
test_number_inactive = 5


k_effective_sample, k_effective_std_sample = simulation_reflective_boundaries(box, test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs)


 k effective  1.7134615384615888+- 0.10471780124685057
 k effective  1.4999999999999714+- 0.10936975987863616
outside
5.000059362191159  4.999948646191761 -0.9747808458087811
 k effective  1.7684509090909823+- 0.1252007630939633


In [150]:
# one medium slab

In [151]:
x_5 = Plane(0, 1, 0, 500)
x_m5 = Plane(0, 1, 0, -500)

y_5 = Plane(1, 0, 0, 1.8)
y_m5 = Plane(1, 0, 0, -1.8)



z_5 = Plane(0, 0, 1, 500)
z_m5 = Plane(0, 0, 1, -500)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab_2 = Cell(surfaces, signs)

In [152]:
test_number_of_particles = 20
test_number_interations = 100
test_number_inactive = 70


k_effective_sample, k_effective_std_sample = simulation_black_boundaries(slab_2,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs)


 k effective  1.0814066163064195+- 0.018487826995184507
 k effective  1.0816857440167826+- 0.0151764664928003
 k effective  1.123620779220654+- 0.028277802358220198
 k effective  1.0225689676693126+- 0.03730449115608671
 k effective  1.1212920237310071+- 0.03877446801092161
 k effective  1.0398171428572327+- 0.0386632775861653
 k effective  1.1210234766554619+- 0.0393064672003735
 k effective  1.0758831003810372+- 0.036804857795694305
 k effective  1.037846938775651+- 0.03720456571578161
 k effective  1.15210512344039+- 0.04209210233927676
 k effective  1.0344456886898592+- 0.04250620790140905
 k effective  1.0867342166797431+- 0.040754365990061055
 k effective  1.075398915569234+- 0.039170975946200254
 k effective  1.0872812135354197+- 0.03780342979019694
 k effective  1.0617797810687715+- 0.03679710808331929
 k effective  1.1286664920097205+- 0.03763838531682058
 k effective  1.029370769616565+- 0.03853225549770612
 k effective  1.1041451867436654+- 0.03789970339235062
 k effective  

In [154]:
test_cylinder = ZCylinder(1, 5, 5)

plane_5 = Plane(0, 0, 1, 5)
plane_6 = Plane(0, 0, 1, -5)

surfaces = [test_cylinder, plane_5, plane_6]
signs = [-1, -1, +1]

cylinder = Cell(surfaces, signs)



In [ ]:
test_number_of_particles = 20
test_number_interations = 100
test_number_inactive = 70


k_effective_sample, k_effective_std_sample = simulation_black_boundaries(slab_2,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs)


In [347]:
test_cylinder = ZCylinder(1, 5, 5)

plane_5 = Plane(0, 0, 1, 5)
plane_6 = Plane(0, 0, 1, -5)

surfaces = [test_cylinder, plane_5, plane_6]
signs = [-1, -1, +1]

cylinder = Cell(surfaces, signs)
new_particles = simulation(cylinder, 36, test_number_interations, test_number_of_particles, test_number_inactive, pu_23_cs)


2
outside
-4.796990095430157  -1.029830583837203 4.999925058241048
outside
-3.305445927506881  -2.819743969498402 4.999900248070012
+++++++++++++++++++++++++++++++++++++++++++++
after splitting 
[]
_____________________
 sum 0
 initial_size  2
after normalisation 
[]
_____________________
+++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++
after splitting 
[]
_____________________
 sum 0
 initial_size  2
after normalisation 
[]
_____________________
+++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++
after splitting 
[]
_____________________
 sum 0
 initial_size  2
after normalisation 
[]
_____________________
+++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++
after splitting 
[]
_____________________
 sum 0
 initial_size  2
after normalisation 
[]
_____________________
+++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++
after 

NameError: name 'current_flux' is not defined

In [613]:
test_cylinder = ZCylinder(1, 5, 5)

plane_5 = Plane(0, 0, 1, 5)
plane_6 = Plane(0, 0, 1, -5)

surfaces = [test_cylinder, plane_5, plane_6]
signs = [-1, -1, +1]

cylinder = Cell(surfaces, signs)
new_particles = simulation(cylinder, 12, test_number_interations, test_number_of_particles, test_number_inactive, pu_23_cs)


8
%%%%%%%%%%%%%%%%%%%%%
direction
0.13988039460510387 -0.8555610095727597 -0.4984464204946812
coordinates
-3.0  -3.0 -3.0
%%%%%%%%%%%%%%%%%%%%%
++++++++++++++++++++++
direction
0.13988039460510387 -0.8555610095727597 -0.4984464204946812
coordinates
-3.0  -3.0 -3.0
--------------------------
direction
0.13988039460510387 -0.8555610095727597 0.21868563128447344
coordinates
-2.438874360757489  -6.432055076501522 -4.999501553579505
%%%%%%%%%%%%%%%%%%%%%
direction
0.9774246565077324 0.031226923780914875 -0.20896392052677984
coordinates
-2.438874360757489  -6.432055076501522 -4.999501553579505
%%%%%%%%%%%%%%%%%%%%%
++++++++++++++++++++++
direction
0.9774246565077324 0.031226923780914875 -0.20896392052677984
coordinates
-2.438874360757489  -6.432055076501522 -4.999501553579505
--------------------------
direction
0.9774246565077324 0.031226923780914875 -1.745885392488685
coordinates
-2.4375203120184397  -6.43201181712957 -4.999791036079473
%%%%%%%%%%%%%%%%%%%%%
direction
0.46897867607701077 0

AssertionError: 

In [168]:
x_5 = Plane(0, 1, 0, 3)
x_m5 = Plane(0, 1, 0, -3)

y_5 = Plane(1, 0, 0, 3)
y_m5 = Plane(1, 0, 0, -3)



surfaces = [x_5, x_m5, y_5, y_m5]
signs = [-1, +1, -1, +1]
club = Cell(surfaces, signs)
new_particles = simulation(cylinder, 10, test_number_interations, test_number_of_particles, test_number_inactive, pu_23_cs)


8000
3335
10912


In [42]:
3.063725 + 1.478450

4.542175